In [1]:
from itertools import chain

In [2]:
from IPython.display import clear_output

In [3]:
import os
import shutil

In [4]:
import subprocess
from datetime import date as ddate
from datetime import timedelta

from dateutil import relativedelta

In [5]:
import numpy as np
import pandas as pd
import xarray as xr

In [6]:
import dask
from dask import compute, delayed
from dask.diagnostics import ProgressBar
from dask.distributed import Client

In [7]:
dask.config.set(
    {
        "distributed.worker.memory.target": False,
        "distributed.worker.memory.spill": False,
    }
)

In [8]:
client = Client(n_workers=12, threads_per_worker=1, processes=True)

In [9]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 125.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37183,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 125.57 GiB
Comm: tcp://127.0.0.1:42769,Total threads: 1
Dashboard: http://127.0.0.1:40019/status,Memory: 10.46 GiB
Nanny: tcp://127.0.0.1:42267,


### parameters for papermill 

In [10]:
ndays_agg = 30

In [11]:
infile = f"/home/nicolasf/data/MSWEP/merged_1979_2020_noleap_runsum{ndays_agg}.nc"

In [12]:
# dates = pd.date_range(start='1992-01-01', end='2017-12-31', freq='1D')

# sdates = pd.Series(dates, index=dates, name='time')

# sdates.index.name = 'time'

# sdates = sdates.to_xarray()

# sdates = sdates.convert_calendar('noleap')

# sdates = sdates.to_dataframe()

# sdates.head()

In [13]:
clim_start = 1993
clim_end = 2016

In [14]:
half_window = 7

In [15]:
dates_dummy = pd.date_range(start="1999-01-01", end="1999-12-31")

In [16]:
cdo = "/home/nicolasf/mambaforge/envs/CDO/bin/cdo"

In [ ]:
for date in dates_dummy[52:]: 

    d_l = []

    for y in np.arange(clim_start, clim_end + 1):
        d = pd.date_range(
            start=ddate(y, date.month, date.day) - timedelta(days=half_window),
            end=ddate(y, date.month, date.day) + timedelta(days=half_window),
        )
        
        ### here convert to noleap calendar ...
        
        d = d[~d.is_leap_year]
        
        d_l.append(d.to_list())

    d_l = list(chain(*d_l))

    d_l = [f"{x:%Y-%m-%d}" for x in d_l]

    outfiles = []

    for d in d_l:

        outfile = f"/home/nicolasf/data/MSWEP/tmp_{date.day_of_year:03d}_{d}.nc"
        outfiles.append(outfile)
        cmd = f"{cdo} seldate,{d} {infile} {outfile}"
        # print(cmd)
        subprocess.call(cmd, shell=True)
        clear_output()

    dset = xr.open_mfdataset(outfiles)

    dset = dset.drop("time_bnds")

    dset = dset.chunk({"time": -1, "lat": 10, "lon": 10})

    dsetm = dset.mean("time")

    dsetq = dset.quantile(
        q=[0.1, 0.2, 0.25, 0.3, 0.3333, 0.4, 0.5, 0.6, 0.6666, 0.7, 0.75, 0.8, 0.9], dim="time"
    )

    with ProgressBar():
        dsetm = dsetm.compute()
        dsetq = dsetq.compute()

    dsetm.to_netcdf(
        f"/media/nicolasf/END19101/ICU/data/glo2ho/MSWEP280/Past/Daily/SP/climatologies/clim_{clim_start}_{clim_end}_average_{ndays_agg}_runsum_DOY_{date.day_of_year}.nc"
    )

    dsetq.to_netcdf(
        f"/media/nicolasf/END19101/ICU/data/glo2ho/MSWEP280/Past/Daily/SP/climatologies/clim_{clim_start}_{clim_end}_quantiles_{ndays_agg}_runsum_DOY_{date.day_of_year}.nc"
    )

    for f in outfiles:
        os.remove(f)

    dset.close()
    dsetq.close()
    dsetm.close()